In [12]:
#from pcaspy import Driver, SimpleServer
import paramiko
#from scp import SCPClient
import os
from pathlib import PureWindowsPath
from epics import caget, caput
#from pcaspy import Driver, SimpleServer
global data_path
global data_destination, current_cycle, current_sample, proposal_id, run_id


prefix = 'XF:16IDC-ES{HPLC}'


# Define PVs and their properties
pvdb = {
    'REMOTE_HOST': {'type': 'string'},
    'USERNAME': {'type': 'string'},
    'PASSWORD': {'type': 'string'},
    'YEAR': {'type': 'string'},
    'MONTH': {'type': 'string'},
    'FILE_TYPE': {'type': 'string'},
    'FILE_NAME': {'type': 'string'},
    'FILE_SOURCE': {'type': 'string'},
    'DEST_PATH': {'type': 'string'},
    'SampleName': {'type': 'string'},
    'TRANSFER': {'type': 'int', 'value': 0}  # Trigger PV
}


class MyDriver(Driver):
    def __init__(self):
        super(MyDriver, self).__init__()
        self.setParam("USERNAME", 'xf16id')
        self.setParam("PASSWORD", 'xf16id')
        self.windows_data_dir=PureWindowsPath("C:/Users/xf16id/CDSProjects/HPLC")
        self.destination_directory = "/nsls2/data/lix/legacy/HPLC/Agilent/"
        self.setParam("DEST_PATH", self.destination_directory)
        self.setParam('FILE_SOURCE', self.windows_data_dir)
        #self.setParam("SampleName", current_sample)
    
    
    def move_hplc_files(self, src_path, dest_dir, username, remote_host, SampleName):
        
        try:
            # Set up SSH and SCP
            ssh = paramiko.SSHClient()
            #ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
            ssh.connect(remote_host, username=username)
            if not os.path.isfile(f"{src_path}+/{SampleName}":
                print (f" Source file {src_path} does not exist!")
                return
            if not os.path.exists(dest_dir):
                os.makedirs(dest_dir)
                print(f" Created directory {dest_dir}!")
                filename = os.path.basename(src_path)
                dest_path = os.path.join(dest_dir, filename)
                shutil.move(src_path, dest_path)
                print(f" Moved ADF file '{src_path}' to '{dest_path}'.")

            with SCPClient(ssh.get_transport()) as scp:
                scp.get(remote_path, dest_path)
            print(f"File {remote_file} successfully copied to {dest_path}")
            self.setParam('TRANSFER', 0)  # Reset trigger after successful transfer

        except Exception as e:
            print(f"Failed to copy file: {e}")

        finally:
            ssh.close()
       
        


    def dynamic_scp_transfer(self, remote_host, username, password, dest_path, **kwargs):
        """Handles SCP transfer with parameters from EPICS PVs."""
        remote_dir = f"{data_path}/CDSProjects/HPLC/{kwargs.get('SampleName')}"
        remote_file = kwargs.get('SampleName')
        remote_path = os.path.join(remote_dir, remote_file)
        print(remote_path)

        try:
            # Set up SSH and SCP
            ssh = paramiko.SSHClient()
            ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
            ssh.connect(remote_host, username=username, password=password)
            with SCPClient(ssh.get_transport()) as scp:
                scp.get(remote_path, dest_path)
            print(f"File {remote_file} successfully copied to {dest_path}")
            self.setParam('TRANSFER', 0)  # Reset trigger after successful transfer

        except Exception as e:
            print(f"Failed to copy file: {e}")

        finally:
            ssh.close()

    def write(self, reason, value):
        """Handles PV writes, especially the TRANSFER trigger PV."""
        self.setParam(reason, value)

        # If the TRANSFER PV is written to 1, initiate the SCP transfer
        if reason == 'TRANSFER' and value == 1:
            # Get all PV values needed for SCP transfer
            src_path = self.getParam('FILE_Source')
            username = self.getParam('USERNAME')
            password = self.getParam('PASSWORD')
            dest_path = self.getParam('DEST_PATH')
            SampleName = self.getParam('SampleName')
            remote_host = self.setParam('REMOTE_HOST','10.66.123.226')

            # Call the SCP transfer function with parameters from PVs
            self.move_hplc_files(src_path=src_path, dest_path=dest_path, username=username, password=password, remote_host=remote_host, SampleName=SampleName
            )

        return True

# Start the EPICS server and driver
if __name__ == '__main__':
    server = SimpleServer()
    server.createPV('MYIOC:', pvdb)
    driver = MyDriver()

    print("IOC is running. Modify TRANSFER PV to start SCP transfer.")

    while True:
        server.process(0.1)


SyntaxError: invalid syntax (716853338.py, line 50)

In [2]:
!pip install pcaspy
from pcaspy import Sim

Traceback (most recent call last):
  File "/bin/pip", line 7, in <module>
    from pip._internal.cli.main import main
ModuleNotFoundError: No module named 'pip'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/bin/pip", line 10, in <module>
    from pip._internal.main import main
ModuleNotFoundError: No module named 'pip'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/bin/pip", line 17, in <module>
    from pip._internal import main
ModuleNotFoundError: No module named 'pip'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/bin/pip", line 20, in <module>
    from pip import main
ModuleNotFoundError: No module named 'pip'


ModuleNotFoundError: No module named 'pcaspy'

In [3]:
def scp_transfer(self, remote_host, username, dest_path, **kwargs):
        """Handles SCP transfer with parameters from EPICS PVs."""
        #remote_dir = f"/base/path/{kwargs.get('year')}/{kwargs.get('month')}/{kwargs.get('file_type')}"
        remote_file_dir = kwargs.get('current_sample')
        remote_file_adf = kwargs.get('current_sample')
        remote_dir=os.PureWindowsPath('C:/Users/xf16id/CDSProjects/HPLC/')
        remote_path = os.PureWindowsPath.join(remote_dir, remote_file, remote_file_adf)
        print(f"fetching from {remote_path}... ")

        try:
            ssh_key = str(pathlib.Path.home())+"/.ssh/id_rsa.pub"
            if not os.path.isfile(ssh_key):
                raise Exception(f"{ssh_key} does not exist!")
            cmd = ["scp", "-i", ssh_key, f"xf16id@{windows_ip}:{remote_path}", "/nsls2/data/lix/legacy/HPLC/Agilent/"]
            subprocess.run(cmd, check=True, universal_newlines=True)
            time.sleep(1)
            #ssh = paramiko.SSHClient()
            #ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
            #ssh.connect(remote_host, username=username, password=password)
            #with SCPClient(ssh.get_transport()) as scp:
             #   scp.get(remote_path, dest_path)
            print(f"File {remote_file} successfully copied to {dest_path}")
            self.setParam('TRANSFER', 0)  # Reset trigger after successful transfer

        except Exception as e:
            print(f"Failed to copy file: {e}")

        finally:
            ssh.close()

In [13]:
import pathlib

In [19]:
def scp_transfer(dest_path, **kwargs):
        """Handles SCP transfer with parameters from EPICS PVs."""
        #remote_dir = f"/base/path/{kwargs.get('year')}/{kwargs.get('month')}/{kwargs.get('file_type')}"
        remote_file_dir = kwargs.get('current_sample')
        remote_file_adf = kwargs.get('current_sample')+'.ADF'
        #remote_file_adf= str(remote_file +'.ADF')
        remote_dir='C:/Users/xf16id/CDSProjects/HPLC/'
        remote_path = os.path.join(remote_dir, remote_file_dir, remote_file_adf)
        print(f"fetching from {remote_path}... ")
'''
        try:
            ssh_key = str(pathlib.Path.home())+"/.ssh/id_rsa.pub"
            if not os.path.isfile(ssh_key):
                raise Exception(f"{ssh_key} does not exist!")
            cmd = ["scp", "-i", ssh_key, f"xf16id@{windows_ip}:{remote_path}", "/nsls2/data/lix/legacy/HPLC/Agilent/"]
            subprocess.run(cmd, check=True, universal_newlines=True)
'''

'\n        try:\n            ssh_key = str(pathlib.Path.home())+"/.ssh/id_rsa.pub"\n            if not os.path.isfile(ssh_key):\n                raise Exception(f"{ssh_key} does not exist!")\n            cmd = ["scp", "-i", ssh_key, f"xf16id@{windows_ip}:{remote_path}", "/nsls2/data/lix/legacy/HPLC/Agilent/"]\n            subprocess.run(cmd, check=True, universal_newlines=True)\n'

In [14]:
import os

In [20]:
current_sample="test"

In [21]:
print(current_sample+'.ADF')

test.ADF


In [23]:
scp_transfer(dest_path=".", current_sample=current_sample)

fetching from C:/Users/xf16id/CDSProjects/HPLC/test/test.ADF... 


In [15]:
import socket
import numpy as np
from time import sleep
import signal
import itertools
import os

In [36]:
from enum import Enum

class data_file_path(Enum):
    old_gpfs = '/GPFS/xf16id/exp_path'
    lustre_legacy = '/nsls2/data/lix/legacy'
    lustre_asset = '/nsls2/data/lix/asset'
    lustre_proposals = '/nsls2/data/lix/proposals'
    gpfs = '/nsls2/xf16id1/data'
    gpfs_experiments = '/nsls2/xf16id1/experiments'
    ramdisk = '/ramdisk'

current_cycle = '2024-3'
#pilatus_data_dir = data_file_path.lustre_legacy.value
pilatus_data_dir = f"{data_file_path.ramdisk.value}/hdf"
hplc_data_dir = f"{data_file_path.lustre_legacy.value}/HPLC"
hplc_data_dest = f"{hplc_data_dir}/{current_cycle}/{proposal_id}/{run_id}/"
xspress3_data_dir = pilatus_data_dir # "/home/xspress3/lix"  # 
data_destination = data_file_path.lustre_legacy.value  # this is where all IOC data files should eventually go
#proc_destination = data_file_path.lustre_proposals.value
proc_destination = data_file_path.lustre_legacy.value
procdir_prefix = "pass-"  

bl_comm_proposal = "314980"

import sys
shared_path = "/nsls2/software/mx/lix/pylibs"
sys.path = [shared_path]+sys.path

In [51]:
###standard PVs to set:
caput('XF:16IDC-ES{HPLC}REMOTE_HOST', '10.66.123.226')
caput('XF:16IDC-ES{HPLC}USER_NAME', 'xf16id')
caput('XF:16IDC-ES{HPLC}PASSWORD', 'xf16id')
caput('XF:16IDC-ES{HPLC}DEST_PATH', 'C:\Users\xf16id\CDSProjects\')

**** The executable "caRepeater" couldn't be located
**** because of errno = "Permission denied".
**** You may need to modify your PATH environment variable.
**** Unable to start "CA Repeater" process.


cannot connect to XF:16IDC-ES{HPLC}REMOTE_HOST


CA client library is unable to contact CA repeater after 50 tries.
Silence this message by starting a CA repeater daemon
or by calling ca_pend_event() and or ca_poll() more often.


In [8]:
def run_hplc_from_spreadsheet(spreadsheet_fn, batchID, sheet_name='Samples', exp=1, shutdown=False):
    global proposal_id, run_id, current_cycle, hplc_data_dir
    batch_fn = '/nsls2/data/lix/legacy/HPLC/Agilent/sequence_table.csv'
    samples, valve_position = CreateAgilentSeqFile(spreadsheet_fn, batchID, sheet_name=sheet_name)
    print(f"HPLC sequence file has been created in: {batch_fn}")
    print("Make sure you have selected the correct valve position for column type!")
    input("Please start Sequence in Agilent software by importing sequence_table.csv (under sequence tab), click run, then come back to this machine and then hit enter:")
    for sn in samples.keys():
        caput('XF:16IDC-ES{HPLC}SampleName', sn)
        caput('XF:16IDC-ES{HPLC}TRANSFER', 1)
        #caput('XF:16IDC-ES{HPLC}SampleNamef"{sn}"')
        VV.switch_10port_valve(pos=samples[sn]["md"]["Valve Position"])
        print(f"Switching valve to position {samples[sn]['md']['Valve Position']}!")
        print(f"collecting data for {sn} ...")

            #print(f"Switching to valve position {pos}!")
            #switch_10port_valve(pos=pos)
        # for hardware multiple trigger, the interval between triggers is slightly longer
        #    than exp. but this extra time seems to fluctuates. it might be safe not to include
        #    it in the caulcation of nframes
        
        collect_hplc(sn, exp=exp, nframes=int(samples[sn]["acq time"]*60/exp), md={'HPLC': samples[sn]['md']})   
        uid=db[-1].start['uid']
        send_to_packing_queue(uid, "HPLC")
    pil.use_sub_directory()    
    print('Sequence collection finished for %s from %s' % (batchID,spreadsheet_fn))
    

In [9]:
def get_IOC_HPLCpath(ioc_name, substitute_path=None):
    global current_sample
    if data_path=="":
        print("login first to specify data path:")
        login()
    if substitute_path:
        return data_path.replace(data_destination, substitute_path)%ioc_name
    else:
        return data_path%ioc_name

In [11]:
def collect_hplc(sample_name, exp, nframes,md=None):
    _md = {"experiment": "HPLC"}
    _md.update(md or {})
    change_sample(sample_name)
    hplc_data_dir=get_IOC_HPLCpath(ioc_name, substitute_path=None)
    pil.use_sub_directory(sample_name)
    sol.select_flow_cell('middle')
    pil.set_trigger_mode(PilatusTriggerMode.ext_multi)
    pil.set_num_images(nframes)
    pil.exp_time(exp)
    update_metadata()
      
    #sol.ready_for_hplc.set(1)
    while sol.hplc_injected.get()==0:
        if sol.hplc_bypass.get()==1:
            sol.hplc_bypass.put(0)
            break
        sleep(0.2)

    #sol.ready_for_hplc.set(0)
    start_monitor([em1,em2], rate=4)
    RE(monitor_during_wrapper(ct([pil], num=nframes, md=_md), [em1.ts.SumAll, em2.ts.SumAll]))
    sd.monitors = []
    pil.use_sub_directory()
    change_sample()

In [41]:
def HPLC_mover():
    '''
    Moves ADF file from HPLC windows server to lustre.
    '''
    global current_sample, current_cycle, proposal_id, run_id, hplc_data_dest
    if run_id == "":
        print("Login first to specify data path and hplc path")
        login()
    if hplc_data_dest:
        hpath=os.makedirs(f"{hplc_data_dest}/{current_sample}/")
        print(f"Creating HPLC data destination at {hpath}")
    return hpath
    

In [4]:
import os
import pathlib

In [ ]:
##copies UV files (ADF) from windows machine to lustre directory.  Will eventually go to assets)
def get_UV_files(windows_ip="10.66.123.226"):
    ssh_key = str(pathlib.Path.home())+"/.ssh/id_rsa.pub"
    windows_path=
    if not os.path.isfile(ssh_key):
        raise Exception(f"{ssh_key} does not exist!")
    cmd = ["scp", "-i", ssh_key, f"xf16id@{windows_ip}:"
    
    
    

In [ ]:
if msg is None:
        ssh_key = str(pathlib.Path.home())+"/.ssh/ot2_ssh_key"
        if not os.path.isfile(ssh_key):
            raise Exception(f"{ssh_key} does not exist!")

        cmd = ["ssh", "-i", ssh_key, "-o", f"port={ot2_port}",
               f"root@{ot2_ip}", "/var/lib/jupyter/notebooks/check_deck_config.py", 
               "-h", holder_slots, "-p", plate_slots]

        ret = subprocess.run(cmd, capture_output=True)
        msg = ret.stdout.decode()

        if ret.returncode:
            print(msg)
            print(ret.stderr.decode())
            raise Exception("error executing check_deck_config.py")
            
    ldict = json.loads(msg.split("*****")[-1])

In [ ]:
get_IOC_HPLCpath(ioc_name=hplc_ioc)

In [23]:
def make_filename(self):
        ''' replaces FileStorePluginBase.make_filename()
        Returns
        -------
        filename : str
            The start of the filename
        read_path : str
            Path that ophyd can read from
        write_path : str
            Path that the IOC can write to
        '''
        global current_sample

        filename = f"{current_sample}_{self.parent.detector_id}"
        write_path = get_IOC_datapath(self.parent.name, self.data_dir) 
        if not self.use_ioc_path:
            write_path = str(Path(self.data_dir))+'/'   # Xspress3 IOC cannot create subdirectories
        if self.sub_directory:
            write_path += f"/{self.sub_directory}"
        read_path = write_path # might want to handle this differently, this shows up in res/db
        return filename, read_path, write_path

In [25]:
from pathlib import Path

In [39]:
path

In [44]:
def CreateAgilentSeqFile(spreadsheet_fn, batchID, sheet_name='Samples'):
    
    """
    Creates the agilent sequence file from the spreadsheet. User input is batchID, sample_name and method for hplc. The other columns are necessary for generating the proper formatted sequence file
    """
    strFields=["Vial", "Sample Name", "Sample Type", "Acq. method", "Proc. method", "Data file", "Buffer", "Valve Position"]
    numFields=["Volume"]
    dd=parseSpreadsheet(spreadsheet_fn, sheet_name=sheet_name, return_dataframe=False)
    print(f"this is {dd}")
    autofillSpreadsheet(dd, fields=["batchID"])
    print(dd['batchID'].keys())
    print(dd['batchID'].values())
    print(dd['Valve Position'].values())
    
    if proposal_id is None or run_id is None:
        print("need to login first ...")
        login()
    
    ridx=[i for i in dd['batchID'].keys() if dd['batchID'][i]==batchID]
    print(f"{ridx} this is ridx")

    
    samples = {}
    dfile = f"{current_cycle}/{proposal_id}/{run_id}/<S>" 
    dd["Data File"] = {}
    dd["Sample Type"] = {}
    valve_position = {}
        
    for i in ridx:
        for f in numFields:
            if not (isinstance(dd[f][i], int or isintance(dd[f][i], float))):
                raise Exception(f"not a numeric value for {f}: {dd[f][i].values()}, replace with number")
        valve_position[i] = dd["Valve Position"][i]
        print(valve_position)
        sn = dd['Sample Name'][i]
        samples[sn] = {"acq time": dd['Run Time'][i], 
                       "valve_position":valve_position[i],
                       "md": {"Column type": dd['Column type'][i],
                              "Injection Volume (ul)": dd['Volume'][i],
                              "Flow Rate (ml_min)": dd['Flow Rate'][i],
                              "Sample buffer":dd['Buffer'][i],
                              "Valve Position":valve_position[i]}
                      }
        dd["Data File"][i] = dfile
        dd["Valve Position"][i]
        dd["Sample Type"][i] = "Sample"
    sequence_path="/nsls2/data/lix/legacy/HPLC/Agilent/"
    df=pd.DataFrame.from_dict(dd, orient='columns')
    df[df['batchID']==batchID].to_csv(f"{sequence_path}sequence_table.csv", index=False, encoding="ASCII",
                    columns=["Vial", "Sample Name", "Sample Type", "Volume", "Inj/Vial", "Acq. method", "Proc. method", "Data File" ])

    return samples, valve_position  


In [24]:
%run -i 25-hplc_testing.py

Loading 25-hplc_testing.py...
10.0


NameError: name 'Enum' is not defined

In [25]:
import Enum

ModuleNotFoundError: No module named 'Enum'

In [36]:
def check_sample_name(sample_name, ioc_name="pil1M", sub_dir=None, check_for_duplicate=True, check_dir=False):    
    if len(sample_name)>42:  # file name length limit for Pilatus detectors
        print("Error: the sample name is too long:", len(sample_name))
        return False
    l1 = re.findall('[^:._A-Za-z0-9\-]', sample_name)
    if len(l1)>0:
        print("Error: the file name contain invalid characters: ", l1)
        return False

    if check_for_duplicate:
        f_path = data_path%ioc_name
        if sub_dir is not None:
            f_path += ('/'+sub_dir+'/')
        #if PilatusFilePlugin.froot == data_file_path.ramdisk:
        #    f_path = data_path.replace(data_file_path.gpfs.value, data_file_path.ramdisk.value)
        if check_dir:
            fl = glob.glob(f_path+sample_name)
        else:
            fl = glob.glob(f_path+sample_name+"_000*")
        if len(fl)>0:
            print(f"Error: name already exists: {sample_name} at {f_path}")
            return False

    return True

<>:5: SyntaxWarning: invalid escape sequence '\-'
<>:5: SyntaxWarning: invalid escape sequence '\-'
/tmp/ipykernel_3832360/2999784052.py:5: SyntaxWarning: invalid escape sequence '\-'
  l1 = re.findall('[^:._A-Za-z0-9\-]', sample_name)


In [26]:

def CreateAgilentSeqFile(spreadsheet_fn, batchID, sheet_name='Samples'):
    
    """
    Creates the agilent sequence file from the spreadsheet. User input is batchID, sample_name and method for hplc. The other columns are necessary for generating the proper formatted sequence file
    """
    strFields=["Vial", "Sample Name", "Sample Type", "Acq. method", "Proc. method", "Data file", "Buffer", "Valve Position"]
    numFields=["Volume"]
    dd, df=parseSpreadsheet(spreadsheet_fn, sheet_name=sheet_name, return_dataframe=True)
    print(f"{df}")
    autofillSpreadsheet(dd, fields=["batchID"])
    #print(dd['batchID'].keys())
    #print(dd['batchID'].values())
    #print(dd['Valve Position'].values())
    
    if proposal_id is None or run_id is None:
        print("need to login first ...")
        login()
    
    ridx=[i for i in dd['batchID'].keys() if dd['batchID'][i]==batchID]
    #print(f"{ridx} this is ridx")

    
    samples = {}
    dfile = f"{run_id}/<S>" 
    dd["Data File"] = {}
    dd["Sample Type"] = {}
    valve_position = {}
    Injection_Volume = []
        
    for i in ridx:
        """
        check for duplicates
        """
        check_sample_name(dd['Sample Name'][i])
        for f in numFields:
            if not (isinstance(dd[f][i], int or isintance(dd[f][i], float))):
                raise Exception(f"not a numeric value for {f}: {dd[f][i].values()}, replace with number")
        valve_position[i] = dd["Valve Position"][i]
        print(valve_position)
        sn = dd['Sample Name'][i]
        samples[sn] = {"acq time": dd['Run Time'][i], 
                       "valve_position":valve_position[i],
                       "md": {"Column type": dd['Column type'][i],
                              "Injection Volume (ul)": dd['Volume'][i],
                              "Flow Rate (ml_min)": dd['Flow Rate'][i],
                              "Sample buffer":dd['Buffer'][i],
                              "Valve Position":valve_position[i]}
                      }
        Vial_position = dd["Vial"][i]
        print(Vial_position)
        Method = dd['Acq. method'][i]
        Injection_Volume = dd['Volume'][i]
        dd["Data File"][i] = dfile
        dd["Valve Position"][i]
        dd["Sample Type"][i] = "Sample"
    sequence_path="/nsls2/data/lix/legacy/HPLC/Agilent/"
    df=pd.DataFrame.from_dict(dd, orient='columns')
    df[df['batchID']==batchID].to_csv(f"{sequence_path}sequence_table.csv", index=False, encoding="ASCII",
                    columns=["Vial", "Sample Name", "Sample Type", "Volume", "Inj/Vial", "Acq. method", "Proc. method", "Data File" ])

    return samples, valve_position, Injection_Volume, Method, Vial_position

In [27]:
proposal_id="123456"
run_id="78912"

In [35]:
a,b,c,d,e=CreateAgilentSeqFile("HPLC_Sep19.xlsx", batchID="a1")

  batchID     Vial           Sample Name  Volume  \
0      a1  D1F-G01    Lysozyme_5mg_sep19      50   
1      a2  D1F-H01                  DDM0      50   
2      a1  D1F-H01    Lysozyme_5mg_sep19      50   
3      a1  D1F-B02  Lysozyme_5mg_sep19_V      50   
4      a1  D1F-D02  Lysozyme_5mg_sep19_E      80   

                                              Buffer  \
0  25mMHepes_pH8.0_50mM_Potassium_Acetate_1mMIP6_...   
1                                               DDM0   
2  25mMHepes_pH8.0_50mM_Potassium_Acetate_1mMIP6_...   
3  25mMHepes_pH8.0_50mM_Potassium_Acetate_1mMIP6_...   
4  25mMHepes_pH8.0_50mM_Potassium_Acetate_1mMIP6_...   

                    Acq. method Sample Type  Run Time  Flow Rate  \
0  superdex_5_150_channel_B.amx      Sample        15        0.3   
1  superdex_5_150_channel_B.amx      Sample        15        0.3   
2  superdex_5_150_channel_B.amx      Sample        15        0.3   
3  superdex_5_150_channel_B.amx      Sample        15        0.3   
4  superde

/nsls2/software/common/jupyter/kernel_envs/beta/.pixi/envs/default/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [33]:
data_path="/nsls2/data4/lix/legacy/%s/2025-1/314980/test/"

In [146]:
a['Lysozyme_5mg_sep19']['md']['Injection Volume (ul)']

50

In [151]:
print(e)

D1F-D02


In [19]:
from lixtools.mailin import parseSpreadsheet, autofillSpreadsheet

/nsls2/software/common/jupyter/kernel_envs/beta/.pixi/envs/default/lib/python3.12/site-packages/lixtools/mailin.py:600: SyntaxWarning: invalid escape sequence '\-'
  l1 = re.findall('[^:._A-Za-z0-9\-]', sample_name)


In [21]:
import pandas as pd

In [39]:
df = pd.read_excel("HPLC_sample_template.xlsx")

/nsls2/software/common/jupyter/kernel_envs/beta/.pixi/envs/default/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [42]:
data=df.to_numpy()

In [45]:
data

array([['a1', 'D1F-A01', nan, 'X-ray_only', 50, 'Tris ph 8.0']],
      dtype=object)

In [ ]:
np.savetxt("HPLC_template.csv", data, delimiter=",")

In [22]:
def move_hplc_files(dest_path, proposal_id=None, run_id=None,csv=False, **kwargs):
        UV_file_prefix = str(f"{run_id}")
        remote_file_dir = kwargs.get('current_sample')
        remote_file_adf = kwargs.get('current_sample')+'.ADF'
        remote_file_csv = kwargs.get('current_sample')+'.dx_DAD1E.CSV'
        UV_data = str(UV_file_prefix + "-" +remote_file_csv)
        print(UV_data)
        remote_dir='C:/Users/xf16id/CDSProjects/HPLC/'
        remote_path = os.path.join(remote_dir, remote_file_dir, remote_file_adf)
        remote_path_csv = os.path.join(remote_dir, remote_file_dir, remote_file_csv)
        cmd = ["scp", f"xf16id@{windows_ip}:{remote_path}", f"/nsls2/data/lix/legacy/HPLC/Agilent/{current_cycle}/{proposal_id}/{run_id}"]
        print(f"Waiting to transfer {remote_file_adf}....")
        if csv==True:
            cmd = ["scp", f"xf16id@{windows_ip}:{remote_path_csv}", f"/nsls2/data/lix/legacy/HPLC/Agilent/{current_cycle}/{proposal_id}/{run_id}"]
            print(f"fetching CSV files from {remote_path_csv}")     


In [72]:
move_hplc_files(dest_path="test", proposal_id='123456', run_id='789123', current_sample="test1", csv=True)

789123-test1.dx_DAD1E.CSV
Waiting to transfer test1.ADF....
fetching CSV files from C:/Users/xf16id/CDSProjects/HPLC/test1/test1.dx_DAD1E.CSV


In [63]:
windows_ip="10.66.123.226"
current_cycle="2025-1"

In [23]:
def parseSpreadsheet(infilename, sheet_name=0, strFields=[], return_dataframe=False):
    """ dropna removes empty rows
    """
    converter = {col: str for col in strFields} 
    DataFrame = pd.read_excel(infilename, sheet_name=sheet_name, 
                              converters=converter, engine="openpyxl")
    DataFrame.dropna(axis=0, how='all', inplace=True)
    if return_dataframe:
        return DataFrame.to_dict(), DataFrame
    return DataFrame.to_dict()

In [84]:
import PIL,openpyxl,subprocess,json

In [24]:
def run_hplc_from_spreadsheet(spreadsheet_fn, batchID, sheet_name='Samples', exp=1, shutdown=False):
    batch_fn = '/nsls2/data/lix/legacy/HPLC/Agilent/sequence_table.csv'
    samples, valve_position, Injection_Volume, Method, Vial_position = CreateAgilentSeqFile(spreadsheet_fn, batchID, sheet_name=sheet_name)
    print(f"HPLC sequence file has been created in: {batch_fn}")
    print("Make sure you have selected the correct valve position for column type!")
    input("Please start Sequence in Agilent software by importing sequence_table.csv (under sequence tab), click run, then come back to this machine and then hit enter:")
    for sn in samples.keys():
        VV.switch_10port_valve(pos=samples[sn]["md"]["Valve Position"])
        print(f"Switching valve to position {samples[sn]['md']['Valve Position']}!")
        caput('XF:16IDC-ES:{HPLC}SampleName', sn)
        caput('XF:16IDC-ES{HPLC}Method', Method)
        caput('XF:16IDC-ES{HPLC}Injection_Volume', Injection_Volume)
        caput('XF:16IDC-ES{HPLC}Vial_position', Vial_position)
        print(f"collecting data for {sn} ...")

            #print(f"Switching to valve position {pos}!")
            #switch_10port_valve(pos=pos)
        # for hardware multiple trigger, the interval between triggers is slightly longer
        #    than exp. but this extra time seems to fluctuates. it might be safe not to include
        #    it in the caulcation of nframes
        
        collect_hplc(sn, exp=exp, nframes=int(samples[sn]["acq time"]*60/exp), md={'HPLC': samples[sn]['md']})   
        uid=db[-1].start['uid']
        send_to_packing_queue(uid, "HPLC")
        caput('XF:16IDC-ES{HPLC}TRANSFER', 1)
        
    pil.use_sub_directory()    
    print('Sequence collection finished for %s from %s' % (batchID,spreadsheet_fn))

In [156]:
run_hplc_from_spreadsheet("HPLC_Sep19.xlsx", batchID="a1")

/nsls2/software/common/jupyter/kernel_envs/beta/.pixi/envs/default/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


this is {'batchID': {0: 'a1', 1: 'a2', 2: 'a1', 3: 'a1', 4: 'a1'}, 'Vial': {0: 'D1F-G01', 1: 'D1F-H01', 2: 'D1F-H01', 3: 'D1F-B02', 4: 'D1F-D02'}, 'Sample Name': {0: 'Lysozyme_5mg_sep19', 1: 'DDM0', 2: 'Lysozyme_5mg_sep19_B', 3: 'Lysozyme_5mg_sep19_V', 4: 'Lysozyme_5mg_sep19_E'}, 'Volume': {0: 50, 1: 50, 2: 50, 3: 50, 4: 80}, 'Buffer': {0: '25mMHepes_pH8.0_50mM_Potassium_Acetate_1mMIP6_1mMTCEP_5percent_glycerol', 1: 'DDM0', 2: '25mMHepes_pH8.0_50mM_Potassium_Acetate_1mMIP6_1mMTCEP_5percent_glycerol', 3: '25mMHepes_pH8.0_50mM_Potassium_Acetate_1mMIP6_1mMTCEP_5percent_glycerol', 4: '25mMHepes_pH8.0_50mM_Potassium_Acetate_1mMIP6_1mMTCEP_5percent_glycerol'}, 'Acq. method': {0: 'superdex_5_150_channel_B.amx', 1: 'superdex_5_150_channel_B.amx', 2: 'superdex_5_150_channel_B.amx', 3: 'superdex_5_150_channel_B.amx', 4: 'superdex_5_150_channel_B.amx'}, 'Sample Type': {0: 'Sample', 1: 'Sample', 2: 'Sample', 3: 'Sample', 4: 'Sample'}, 'Run Time': {0: 15, 1: 15, 2: 15, 3: 15, 4: 15}, 'Flow Rate': {

Please start Sequence in Agilent software by importing sequence_table.csv (under sequence tab), click run, then come back to this machine and then hit enter: 


NameError: name 'VV' is not defined

In [29]:
import glob, re